<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>

# 为大规模推理部署模型

## 06 - 高级推理
-------

**目录**

* [简介](#introduction)
* [内部维护](#housekeeping)
* [性能分析器](#performance)
* [模型分析器](#model)
* [CPU 基准测试](#cpu)
* [不同的批量大小](#variable)
* [动态批处理](#dynamic-batching)
* [HTTP 与 gRPC](#protocol)
* [异步推理](#async)
* [共享内存](#shared)
* [小结](#conclusion)

<a id="introduction"></a>
### 简介

我们将在此 notebook 中探索如何使用 Triton 推理服务器执行高级推理。我们将探索性能分析器、模型分析器等工具，了解如何访问性能指标，以及如何使用 GPU、可变批量大小、动态批处理、HTTP 和 gRPC 协议、异步推理和共享内存来优化您的应用程序的延迟和吞吐量。

<a id="housekeeping"></a>
### 内部维护

在进一步操作之前，我们将做一些内部维护，并导入我们将使用的一些客户端库，以及定义我们将在 notebook 中使用的一些变量。

In [ ]:
import numpy as np
import time
import tritonclient.http as tritonhttpclient
import tritonclient.grpc as tritongrpcclient
from tqdm import tqdm


http_url = 'triton:8000'
grpc_url = 'triton:8001'
verbose = False
concurrency = 32
model_version = '1'
triton_http_client = tritonhttpclient.InferenceServerClient(url=http_url, verbose=verbose, concurrency=concurrency)
triton_grpc_client = tritongrpcclient.InferenceServerClient(url=grpc_url, verbose=verbose)
input_dtype = 'FP32'

<a id="performance"></a>
### 性能分析器

优化模型推理性能的关键部分之一是能够在试验不同的优化策略时测量性能的变化。`perf_analyzer` 应用（以前被称为 `perf_client`）为 Triton 推理服务器执行此任务。`perf_analyzer` 包含在客户端示例中，可从多个来源获得。

`perf_analyzer` 应用为您的模型生成推理请求，并测量这些请求的吞吐量和延迟。要获得具有代表性的结果，`perf_analyzer` 测量一个时间窗口内的吞吐量和延迟，然后重复测量直到获得稳定的值。在默认情况下，`perf_analyzer` 使用平均延迟来确保稳定性，但您可以使用 `--percentile` 标志根据该置信度来稳定结果。例如，如果使用 `--percentile=95`，则使用第 95 百分位的请求延迟来稳定结果。

例如，我们可以运行以下任一项来分析模型的性能：

```
perf_analyzer \
  -m simple-tensorflow-model \
  -b 1 \
  --concurrency-range 1:1 \
  --shape input_0:1,224,224,3

perf_analyzer \
  -m simple-pytorch-model \
  -b 1 \
  --concurrency-range 1:1

perf_analyzer \
  -m simple-onnx-model \
  -b 1 \
  --concurrency-range 1:1
  
perf_analyzer \
  -m simple-tensorrt-fp32-model \
  -b 1 \
  --concurrency-range 1:1
  
perf_analyzer \
  -m simple-tensorrt-fp16-model \
  -b 1 \
  --concurrency-range 1:1
```

很遗憾，如果我们在**轮询**模式下部署 Triton 推理服务器，则无法运行 `perf_analyzer`。然而，有关 `perf_analyzer` 的更多详细信息，您可以参阅此处的相关文档：https://github.com/triton-inference-server/server/blob/r20.12/docs/perf_analyzer.md

<a id="model"></a>
### 模型分析器

作为一种工具，Triton 模型分析器会使用性能分析器向模型发送请求，同时测量 GPU 显存和计算资源的利用率。模型分析器特别适用于刻画模型在不同的批处理和模型实例的配置下对 GPU 的显存的需求特征。获得此 GPU 显存使用信息后，您可以更明智地决定如何在保持 GPU 显存容量的同时，在同一 GPU 上合并多个模型。

相关详情请参阅[模型分析器库](https://github.com/triton-inference-server/model_analyzer) 以及[使用 NVIDIA 模型分析器充分提升深度学习推理性能](https://developer.nvidia.com/blog/maximizing-deep-learning-inference-performance-with-nvidia-model-analyzer) 中的详细说明。

<a id="cpu"></a>
### CPU 基准测试

在了解一些先进的推理技术之前，我们先在 CPU 上对其中一个模型执行基准测试。Triton 推理服务器非常灵活，它不仅适用于各种类型的深度学习框架，而且能够支持将模型部署到 CPU 上。如要部署到 CPU，只需将：

```
instance_group [
    {
      count: 1
      kind: KIND_CPU
    }
  ]
```

添加到配置文件中即可。下面，我们将 `simple-pytorch-model` 复制到新的模型目录中并修改模型配置文件，以便 Triton 推理服务器将其部署在 CPU 上。

In [ ]:
!rm -rf models/simple-pytorch-model-cpu/
!cp -R models/simple-pytorch-model/ models/simple-pytorch-model-cpu/

In [ ]:
configuration = """
name: "simple-pytorch-model-cpu"
platform: "pytorch_libtorch"
max_batch_size: 32
instance_group [
    {
      count: 1
      kind: KIND_CPU
    }
  ]
input [
 {
    name: "input__0"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""

with open('models/simple-pytorch-model-cpu/config.pbtxt', 'w') as file:
    file.write(configuration)

In [ ]:
!sleep 45

接下来，我们将完成定义 `InferInput` 和 `InferRequestedOutput` 对象的常见过程，并将数据赋给模型的输入。

In [ ]:
input_name = 'input__0'
input_shape = (1, 3, 224, 224)
output_name = 'output__0'
model_name = 'simple-pytorch-model'

input0 = tritonhttpclient.InferInput(input_name, input_shape, input_dtype)
dummy_data = np.ones(shape=input_shape, dtype=np.float32)
input0.set_data_from_numpy(dummy_data, binary_data=True)

output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=True)

让我们先向部署在 GPU 上 `simple-pytorch-model` 提交 1000 个请求（每个请求批量大小为 1）。

In [ ]:
# note: batch size 1

start_time = time.time()
requests = []
request_count = 1000
for i in tqdm(range(request_count)):
    requests.append(triton_http_client.infer(model_name, model_version=model_version, 
                                             inputs=[input0], outputs=[output]))
end_time = time.time()

In [ ]:
batch_size = 1
print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

接下来，我们将向部署在 CPU 上的 `simple-pytorch-model-cpu` 提交 1000 个相同的请求。两者的差异相当明显！

In [ ]:
model_name = 'simple-pytorch-model-cpu'

# note: feel free to stop running this cell at any time!

start_time = time.time()
requests = []
request_count = 1000
for i in tqdm(range(request_count)):
    requests.append(triton_http_client.infer(model_name, model_version=model_version, 
                                             inputs=[input0], outputs=[output]))
end_time = time.time()

In [ ]:
print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

<a id="variable"></a>
### 不同的批量大小

到目前为止，我们一直处理批量大小为 1 的数据输入。但是，我们通常想发送不同的批量大小，例如 4、8、32 甚至更高。这自然需要权衡延迟和吞吐量。如果批量较大，则处理单个批次可能需要更长时间，从而增加了延迟。但是，由于 GPU 能够处理更多数据，且受网络和 I/O 的限制也较小，因此吞吐量或每秒可处理的示例数量可能会增加。

下面，我们将使用 `simple-tensorrt-fp16-model` 并传递批量大小为 1 的 10000 个请求。我们看到此过程大约需要 45 秒。

In [ ]:
input_name = 'actual_input_1'
input_shape = (1, 3, 224, 224)
output_name = 'output1'
model_name = 'simple-tensorrt-fp16-model'

input0 = tritonhttpclient.InferInput(input_name, input_shape, input_dtype)
dummy_data = np.ones(shape=input_shape, dtype=np.float32)
input0.set_data_from_numpy(dummy_data, binary_data=True)
output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=True)

In [ ]:
# note: batch size 1

start_time = time.time()
requests = []
request_count = 10000
for i in tqdm(range(request_count)):
    requests.append(triton_http_client.infer(model_name, model_version=model_version, 
                                             inputs=[input0], outputs=[output]))
end_time = time.time()

In [ ]:
batch_size = 1
print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

现在，我们将传递 300 个批量大小为 32 的请求。我们看到，通过增加批量大小，平均延迟增加了，但总体吞吐量也提高了。

In [ ]:
input_shape = (32, 3, 224, 224)
input0 = tritonhttpclient.InferInput(input_name, input_shape, input_dtype)
dummy_data = np.ones(shape=input_shape, dtype=np.float32)
input0.set_data_from_numpy(dummy_data, binary_data=True)

In [ ]:
# note: batch size 32

start_time = time.time()
requests = []
request_count = 300
for i in tqdm(range(request_count)):
    requests.append(triton_http_client.infer(model_name, model_version=model_version, 
                                             inputs=[input0], outputs=[output]))
end_time = time.time()

In [ ]:
batch_size = 32
print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

<a id="dynamic-batching"></a>
### 动态批处理

动态批处理是 Triton 的一项功能，它允许服务器组合推理请求，以便动态创建批量任务。创建批量请求通常会提高吞吐量。要启用动态批处理，只需将：

```
dynamic_batching {
    preferred_batch_size: [ 4, 8 ]
    max_queue_delay_microseconds: 100
  }
```

添加到配置文件中即可。`preferred_batch_size property` 表示动态批处理器应尝试创建的批量大小。例如，上述配置可实现首选批量大小为 4 和 8 的动态批处理。

动态批处理程序可以配置为允许请求在调度器中延迟有限的时间，以便让更多的请求加入到动态批量任务中。例如，下方的配置将请求的最大延迟时间设为 100 微秒。

当无法创建首选大小的批量时，`max_queue_delay_microseconds` 属性设置会更改动态批处理器行为。当无法根据可用请求创建首选大小的批量时，只要请求延迟的时间不会超过已配置 `max_queue_delay_microseconds` 的值，动态批处理器就将延迟发送批量。如果在此延迟期间到达新请求，并允许动态批处理程序形成首选批量大小的批量，则立即发送该批量以供推理。如果延迟过期，即使不是首选尺寸，动态批处理器也会如期发送批量。

下面，我们将 `simple-tensorrt-fp16-model` 复制到新的模型目录中并修改模型配置文件，以便 Triton 推理服务器使用动态批处理加以部署。

In [ ]:
!rm -rf models/dynamic-batching-model/
!cp -R models/simple-tensorrt-fp16-model/ models/dynamic-batching-model/

In [ ]:
configuration = """
name: "dynamic-batching-model"
platform: "tensorrt_plan"
dynamic_batching { 
  preferred_batch_size: [ 4, 8, 16, 32 ] 
  max_queue_delay_microseconds: 100 }
max_batch_size: 32
input [
 {
    name: "actual_input_1"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224, 224 ]
  }
]
output {
    name: "output1"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""

with open('models/dynamic-batching-model/config.pbtxt', 'w') as file:
    file.write(configuration)

In [ ]:
!sleep 45

下面，我们将使用 `dynamic-batching-model` 并传递批量大小为 1 的 10000 个请求。

In [ ]:
input_shape = (1, 3, 224, 224)
model_name = 'dynamic-batching-model'

input0 = tritonhttpclient.InferInput(input_name, input_shape, input_dtype)
dummy_data = np.ones(shape=input_shape, dtype=np.float32)
input0.set_data_from_numpy(dummy_data, binary_data=True)

In [ ]:
# note: batch size 1

start_time = time.time()
requests = []
request_count = 10000
for i in tqdm(range(request_count)):
    requests.append(triton_http_client.infer(model_name, model_version=model_version, 
                                             inputs=[input0], outputs=[output]))
end_time = time.time()

In [ ]:
batch_size = 1
print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

<a id="protocol"></a>
### HTTP 与 gRPC

客户端可以使用 HTTP/REST 或 GRPC 协议，或者通过 C API 与 Triton 通信。大部分人都熟悉 HTTP，这是互联网的支柱，而 gRPC 是一款更新的开源远程程序调用系统，最初由 Google 于 2015 年开发而成，使用 HTTP/2 传输和协议缓冲区作为接口描述语言，十分高效易用。

下面，我们使用 `tritonclient.grpc` 模组实例化新的 `InferInput` 对象和 `InferRequestedOutput` 对象，并使用 `tritonclient.grpc.InferenceServerClient` 实例将批量大小为 1 的 10000 个请求发送到我们的 `dynamic-batching-model`。我们可以立即看到，使用略微不同的协议会对延迟和吞吐量产生巨大影响！

In [ ]:
input_shape = (1, 3, 224, 224)
model_name = 'dynamic-batching-model'

input0 = tritongrpcclient.InferInput(input_name, input_shape, input_dtype)
dummy_data = np.ones(shape=input_shape, dtype=np.float32)
input0.set_data_from_numpy(dummy_data)
output = tritongrpcclient.InferRequestedOutput(output_name)

In [ ]:
start_time = time.time()
requests = []
request_count = 10000
for i in tqdm(range(request_count)):
    requests.append(triton_grpc_client.infer(model_name, model_version=model_version, 
                                             inputs=[input0], outputs=[output]))
end_time = time.time()

In [ ]:
print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

<a id="async"></a>
### 异步推理

到目前为止，我们均是以同步方式向 Triton 推理服务器提交了我们的请求。换言之，我们向 Triton 提交请求，Triton 计算并返回结果，然后我们提交下一个请求。但是，如果我们可以提交尽可能多的请求，允许 Triton 将尚未处理的请求排入队列，并在得到计算结果后加以返回，又如何呢？ 这种样式被称为异步推理，可以大幅提升吞吐量。

下面，我们创建一个实用 `callback` 函数来处理异步请求，并借助 `tritonclient.grpc.InferenceServerClient` 实例的 `async_infer` 方法将批量大小为 1 的 10000 个请求提交至 `dynamic-batching-model`。吞吐量的提升令人难以置信！

In [ ]:
from functools import partial


results = []

def callback(user_data, result, error):
    if error:
        user_data.append(error)
    else:
        user_data.append(result)

In [ ]:
start_time = time.time()
async_requests = []
request_count = 10000
for i in tqdm(range(request_count)):
    # Asynchronous inference call.
    async_requests.append(triton_grpc_client.async_infer(model_name=model_name, inputs=[input0], 
                                                         callback=partial(callback, results), 
                                                         outputs=[output]))
end_time = time.time()

In [ ]:
print('Average Latency: ~{} seconds'.format((end_time - start_time) / request_count))
print('Average Throughput: ~{} examples / second'.format(batch_size * request_count / (end_time - start_time)))

In [ ]:
print('Example shape of one example of our output data:', results[0].as_numpy(output_name).shape)

<a id="shared"></a>
### 共享内存

使用系统共享内存和 CUDA 共享内存在客户端库和 Triton 之间传递张量，在某些情况下可以显著提高性能。很遗憾，这个领域超出了本实验的范围，但我们强烈建议感兴趣的人查看以下文档和客户端示例：https://github.com/triton-inference-server/server/blob/r20.12/docs/client_examples.md#system-shared-memory 。

<a id="conclusion"></a>
### 小结

在此 notebook 中，我们探索了如何使用 Triton 推理服务器执行高级推理。我们探索了性能分析器、模型分析器等工具，并了解了如何访问性能指标，以及如何使用 GPU、可变批量大小、动态批处理、HTTP 和 gRPC 协议、异步推理和共享内存优化您应用中的延迟和吞吐量。

我们建议您运行下面的单元进行清理。此操作将释放 GPU 显存，以供实验的其他部分使用。

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>